In [ ]:
import os
os.environ['PGHOST'] = 'wrds-pgdata.wharton.upenn.edu'
os.environ['PGPORT'] = '9737'
os.environ['PGDATABASE'] = 'wrds'
os.environ['PGUSER'] = ''                    # enter your wrds username here
os.environ['PGPASSWORD'] = ''           # enter your wrds password here
import wrds
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from datetime import date

db = wrds.Connection()

In [ ]:
def parse_granularity(gran_str):
    """Convert a granularity string to seconds.
       Valid inputs: 1s, 5s, 15s, 30s, 1m, 2m, 5m, 15m, 30m, 60m, 1h, 1d.
    """
    unit = gran_str[-1]
    value = int(gran_str[:-1])
    if unit == 's':
        return value
    elif unit == 'm':
        return value * 60
    elif unit == 'h':
        return value * 3600
    elif unit == 'd':
        return value * 86400
    else:
        raise ValueError("Invalid granularity")

def get_ohlc_data(db, ticker, year, granularity, start_date=None, end_date=None):
    # Clip the start and end dates to the current year
    if start_date is None:
        start_date = date(year, 1, 1)
    if end_date is None:
        end_date = date(year, 12, 31)

    gran_seconds = parse_granularity(granularity)
    
    query = f"""
        WITH base AS (
            SELECT
                (date + time_m) AS ts,
                ROUND((best_bid + best_ask) / 2, 2) AS price
            FROM
                taqm_{year}.complete_nbbo_{year}
            WHERE
                sym_root = '{ticker}'
                AND date BETWEEN DATE '{start_date}' AND DATE '{end_date}'
                AND time_m >= '09:30:00'
                AND time_m <= '16:00:00'
        ),
        intervals AS (
            SELECT
                ts,
                price,
                timestamp 'epoch' + floor(extract(epoch from ts) / {gran_seconds}) * interval '{gran_seconds} second' AS interval_start
            FROM base
        )
        SELECT
            interval_start AS timestamp,
            (ARRAY_AGG(price ORDER BY ts))[1] AS open,
            MAX(price) AS high,
            MIN(price) AS low,
            (ARRAY_AGG(price ORDER BY ts DESC))[1] AS close
        FROM intervals
        GROUP BY interval_start
        ORDER BY interval_start
    """
    df = db.raw_sql(query)
    return df

def get_multi_year_data(db, ticker, start_date, end_date, granularity):
    years = range(start_date.year, end_date.year + 1)
    dataframes = []

    def task_for_year(year):
        # Truncate range to valid range for this year
        year_start = max(start_date, date(year, 1, 1))
        year_end   = min(end_date, date(year, 12, 31))
        return get_ohlc_data(db, ticker, year, granularity, start_date=year_start, end_date=year_end)

    with ThreadPoolExecutor(max_workers=len(years)) as executor:
        futures = {executor.submit(task_for_year, year): year for year in years}
        
        for future in futures:
            year = futures[future]
            try:
                df_year = future.result()
                dataframes.append(df_year)
                print(f"Data for {year} retrieved successfully.")
            except Exception as e:
                print(f"Error retrieving data for {year}: {e}")

    final_df = pd.concat(dataframes).sort_values(by='timestamp').reset_index(drop=True)
    return final_df


In [21]:
df_final_2 = get_multi_year_data(db, 'AAL', date(2014, 5, 1), date(2017, 5, 1), "5m")
df_final_2.head()

Data for 2014 retrieved successfully.
Data for 2015 retrieved successfully.
Data for 2016 retrieved successfully.
Data for 2017 retrieved successfully.


,timestamp,open,high,low,close
0,2014-05-01 09:30:00,35.81,36.11,35.59,35.66
1,2014-05-01 09:35:00,35.65,35.95,35.46,35.92
2,2014-05-01 09:40:00,35.92,36.08,35.8,36.0
3,2014-05-01 09:45:00,36.0,36.16,35.97,36.14
4,2014-05-01 09:50:00,36.14,36.22,35.73,35.85


## table description

In [8]:
db.describe_table(library='taqm_2018', table='complete_nbbo_2018')

Approximately 47080043353 rows in taqm_2018.complete_nbbo_2018.


,name,nullable,type,comment
0,date,False,DATE,Date of quote
1,time_m,False,TIME,Trade Time
2,time_m_nano,True,SMALLINT,Nanosecond offset of time_m
3,sym_root,False,TEXT,Security symbol root
4,sym_suffix,True,TEXT,Security symbol suffix
5,qu_cond,True,VARCHAR(1),Condition of quote issued
6,natbbo_ind,True,VARCHAR(1),Effect of new quote on NBBO
7,qu_source,True,VARCHAR(1),None
8,nbbo_qu_cond,True,VARCHAR(1),None
9,best_bid,True,NUMERIC,National Best Bid


In [3]:
db.describe_table(library='taqm_2014', table='complete_nbbo_2014')

Approximately 21021089671 rows in taqm_2014.complete_nbbo_2014.


,name,nullable,type,comment
0,date,False,DATE,Date of quote
1,time_m,False,TIME,Trade Time milliseconds (microseconds starting...
2,sym_root,False,TEXT,Security symbol root
3,sym_suffix,True,TEXT,Security symbol suffix
4,qu_cond,True,VARCHAR(1),Condition of quote issued
5,natbbo_ind,True,VARCHAR(1),Effect of new quote on NBBO
6,qu_source,True,VARCHAR(1),None
7,nbbo_qu_cond,True,VARCHAR(1),None
8,best_bid,True,NUMERIC,National Best Bid
9,best_bidsizeshares,True,INTEGER,Maximum Bid Size of all exchanges at National ...


In [5]:
db.describe_table(library='taqm_2014', table='cqm_2014')

Approximately 153407034388 rows in taqm_2014.cqm_2014.


,name,nullable,type,comment
0,date,False,DATE,Date of quote
1,time_m,False,TIME,Time to the millisecond
2,ex,True,VARCHAR(1),Exchange that issued the quote
3,sym_root,False,TEXT,Security symbol root
4,sym_suffix,True,TEXT,Security symbol suffix
5,bid,True,NUMERIC,Bid price
6,bidsiz,True,INTEGER,Bid size in units of trade
7,ask,True,NUMERIC,Ask price
8,asksiz,True,INTEGER,Ask size in units of trade
9,qu_cond,True,VARCHAR(1),Condition of quote issued
